In [ ]:
from ifm3dpy import O3RCamera, FrameGrabber, ImageBuffer
import json
import matplotlib.pyplot as plt
import cv2
import open3d as o3d
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from glob import glob
from PIL import Image
from time import time
%matplotlib qt

In [ ]:
''' Utilities '''

def timer(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {1000*(t2-t1):.0f}ms')
        return result
    return wrap_func

def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 8))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')) + f' {image.shape} {image.shape[1] / image.shape[0]:.2f}'.title())
        plt.imshow(image)
    plt.show()
    
def load_pickle(filename = 'bag'):
    with open(filename, "rb") as fp:
            frames = pickle.load(fp)
    return frames

''' Calibration '''

# @timer
def collect_calibration_points(img):
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        cv2.drawChessboardCorners(img, (9,6), corners2, ret)
        plt.imshow(img)
        
        return (objp, corners)
    return (None, None)

# @timer
def calibrate(objpoints, imgpoints, im_size):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, im_size, None, None)
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, im_size, 0, im_size)
    
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error
    print( "total error: {}".format(mean_error/len(objpoints)) )
    return (mtx, newcameramtx, roi, dist)

def undistort(img, config):
    mtx, newcameramtx, roi, dist = config
    dst = cv2.undistort(img, mtx, dist, newcameramtx)
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

In [13]:
imgs = [cv2.imread(im) for im in glob(r'C:\Users\umilinn\Pictures\Camera Calibration\*.jpg')]

In [14]:
objectPoints = []
imagePoints = []
for img in imgs:
    objp, corners = collect_calibration_points(img)
    if objp is not None:
        objectPoints.append(objp)
        imagePoints.append(corners)

In [15]:
config = calibrate(objectPoints, imagePoints, img.shape[::-1][1:])

total error: 0.037024846355966874


In [60]:
%%time
visualize(image = img, rectified = undistort(img, config))

Wall time: 110 ms


In [17]:
config

(array([[607.43997952,   0.        , 645.96889836],
        [  0.        , 605.58954233, 391.50563407],
        [  0.        ,   0.        ,   1.        ]]),
 array([[606.9654541 ,   0.        , 645.46430289],
        [  0.        , 604.83251953, 391.0162154 ],
        [  0.        ,   0.        ,   1.        ]]),
 (0, 0, 1279, 799),
 array([[-0.35962477,  0.16171162, -0.00081826, -0.00444003, -0.03848117]]))

In [ ]:
frames = load_pickle()

In [ ]:
dep = frames[0][2]

In [33]:
im_size

NameError: name 'im_size' is not defined

In [52]:
dep_mtx = np.array([[128.5714264, 0, 110.11029816],
                    [0, 128.5714264, 85.265068],
                    [0, 0, 1]])
dep_dist = np.array([.50292897, -.0403364, 0.0, -.63675499])
dep_newcameramtx, dep_roi = cv2.getOptimalNewCameraMatrix(dep_mtx, dep_dist, dep.shape[::-1], 0, dep.shape[::-1])
dep_config = (dep_mtx, dep_newcameramtx, dep_roi, dep_dist)

In [55]:
dep_dist

array([ 0.50292897, -0.0403364 ,  0.        , -0.63675499])

In [62]:
plt.imshow(undistort(dep, dep_config))

In [47]:
config

(array([[607.43997952,   0.        , 645.96889836],
        [  0.        , 605.58954233, 391.50563407],
        [  0.        ,   0.        ,   1.        ]]),
 array([[606.9654541 ,   0.        , 645.46430289],
        [  0.        , 604.83251953, 391.0162154 ],
        [  0.        ,   0.        ,   1.        ]]),
 (0, 0, 1279, 799),
 array([[-0.35962477,  0.16171162, -0.00081826, -0.00444003, -0.03848117]]))

In [48]:
dep_config

(array([[128.5714264 ,   0.        , 110.11029816],
        [  0.        , 128.5714264 ,  85.265068  ],
        [  0.        ,   0.        ,   1.        ]]),
 array([[222.53392029,   0.        ,  25.87748873],
        [  0.        , 658.22351074,  83.93408901],
        [  0.        ,   0.        ,   1.        ]]),
 (0, 0, 223, 171),
 array([ 0.50292897, -0.0403364 ,  0.        , -0.63675499]))